In [1]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [2]:
from os import getcwd

In [9]:
class MNIST:
    def __init__(self, export_path, buffer_size=1000, batch_size=32, learning_rate=1e-3, epochs=10):
        self._export_path = export_path
        self._buffer_size = buffer_size
        self._batch_size = batch_size
        self._learning_rate = learning_rate
        self._epochs = epochs
        
        self._build_model()
        self.train_dataset, self.test_dataset = self._prepare_dataset()
        
    def preprocess_fn(self, x):
        # Cast x to tf.float32 using the tf.cast func
        # Also normalize the func between [0, 1]
        x = tf.cast(x, dtype=tf.float32) / 255.0
        return x
    
    def _build_model(self):
        
        self._model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(28, 28, 1), dtype=tf.uint8),
            tf.keras.layers.Lambda(self.preprocess_fn),
            tf.keras.layers.Conv2D(8, 3, padding='same'),
            tf.keras.layers.MaxPool2D(),
            tf.keras.layers.Conv2D(16, 3, padding='same'),
            tf.keras.layers.MaxPool2D(),
            tf.keras.layers.Conv2D(32, 3, padding='same'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128),
            tf.keras.layers.Dense(10)
        ])
        optimizer_fn = tf.keras.optimizers.Adam(lr=self._learning_rate)
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        metrics_list = ['accuracy']
        
        self._model.compile(optimizer_fn, loss=loss_fn, metrics=metrics_list)
        
    def _prepare_dataset(self):
        filePath = f"{getcwd()}/../tmp2"
        
        (train_dataset, test_dataset), _ = tfds.load('mnist',
            split=['train', 'test'],
            data_dir=filePath,
            shuffle_files=True,
            as_supervised=True,
            with_info=True,
        ) # YOUR CODE HERE
        
        return train_dataset, test_dataset
    
    def train(self):
        dataset_tr = self.train_dataset.shuffle(self._buffer_size).batch(self._batch_size)
        self._model.fit(dataset_tr, epochs=self._epochs)
        
    def test(self):
        dataset_te = self.test_dataset.batch(32)
        results = self._model.evaluate(dataset_te)
        
        for name, value in zip(self._model.metrics_names, results):
            print("%s: %.3f" % (name, value))
            
    def export_model(self):
        tf.saved_model.save(self._model, self._export_path)

In [10]:
# Define the training parameters.
args = {'export_path': './saved_model',
        'buffer_size': 1000,
        'batch_size': 32,
        'learning_rate': 1e-3,
        'epochs': 5
}

# Create the mnist object. 
mnist = MNIST(**args)

# Train the model.
mnist.train()

# Save the model.
mnist.export_model()

# Evaluate the trained MNIST model.
mnist.test()

Epoch 1/5
1875/1875 [==============================] - 25s 14ms/step - loss: 3.5574 - accuracy: 0.1508
Epoch 2/5
1875/1875 [==============================] - 22s 12ms/step - loss: 2.3025 - accuracy: 0.0988
Epoch 3/5
1875/1875 [==============================] - 23s 12ms/step - loss: 2.3025 - accuracy: 0.0988
Epoch 4/5
1875/1875 [==============================] - 21s 11ms/step - loss: 2.3025 - accuracy: 0.0988
Epoch 5/5
1875/1875 [==============================] - 21s 11ms/step - loss: 2.3025 - accuracy: 0.0988
INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


    313/Unknown - 2s 6ms/step - loss: 2.3026 - accuracy: 0.0981loss: 2.303
accuracy: 0.098
